In [ ]:
from google.colab import files
uploaded = files.upload()

Saving A_Schwab_Corp_Tx_2024_25.csv to A_Schwab_Corp_Tx_2024_25.csv


In [ ]:
pip install yfinance

In [ ]:
pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display
from dateutil.relativedelta import relativedelta

def load_data(file): #loading data
    df = pd.read_csv(file)


    if 'Symbol' in df.columns and 'Action' in df.columns:
        df = df.rename(columns={
            'Symbol': 'symbol',
            'Action': 'action',
            'Quantity': 'quantity',
            'Price': 'price',
            'Date': 'date',
            'Description': 'description'
        })
    elif 'Ticker' in df.columns and 'Trans Code' in df.columns:
        df = df.rename(columns={
            'Ticker': 'symbol',
            'Trans Code': 'action',
            'Qty': 'quantity',
            'Price': 'price',
            'Trade Date': 'date',
            'Description': 'description'
        })

    return df

def clean_data(df): #cleaning data
    df = df.copy()

    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

    required_cols = ['date', 'symbol', 'action', 'quantity', 'price']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")


    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
    df['price'] = pd.to_numeric(df['price'], errors='coerce')

    df['action'] = df['action'].str.strip().str.upper()
    df['symbol'] = df['symbol'].str.strip().str.upper()
    df['description'] = df['description'].astype(str).str.strip().str.upper()

    df = df.dropna(subset=['date', 'symbol', 'action', 'quantity', 'price'])
    df = df[df['quantity'] > 0]

    return df

def categorize_transaction(row): #categorizing data

    description = str(row.get('description', '')).upper()
    action = row['action'].upper()

    if any(x in description for x in ['PUT', 'CALL', 'OPTION']):
        return 'Options'

    elif any(x in action for x in ['BUY', 'SELL', 'REINVEST']):
        return 'Stocks'
    else:
        return 'Other'

def get_current_stock_price(symbol): #getting current prices

    try:
        ticker = yf.Ticker(symbol)
        hist = ticker.history(period="1d")
        return hist['Close'].iloc[-1] if not hist.empty else 0
    except:
        return 0

def calculate_stock_profits(transactions): #calculating profits of stocks
    transactions = transactions.sort_values('date')
    purchases = []
    realized_pnl = 0

    for _, row in transactions.iterrows():
        action = row['action'].upper()
        quantity = row['quantity']
        price = row['price']
        amount = quantity * price

        if action in ['BUY', 'REINVEST']:
            purchases.append({
                'quantity': quantity,
                'cost_basis': amount,
                'symbol': row['symbol']
            })
        elif action == 'SELL':
            remaining_qty = quantity

            while remaining_qty > 0 and purchases:
                oldest_lot = purchases[0]
                qty_to_sell = min(remaining_qty, oldest_lot['quantity'])


                sale_amount = qty_to_sell * price
                cost_basis = (qty_to_sell / oldest_lot['quantity']) * oldest_lot['cost_basis']
                realized_pnl += sale_amount - cost_basis


                oldest_lot['quantity'] -= qty_to_sell
                oldest_lot['cost_basis'] -= cost_basis
                remaining_qty -= qty_to_sell


                if oldest_lot['quantity'] == 0:
                    purchases.pop(0)


    unrealized_pnl = 0
    for lot in purchases:
        current_price = get_current_stock_price(lot['symbol'])
        market_value = lot['quantity'] * current_price
        unrealized_pnl += market_value - lot['cost_basis']

    return {
        'Realized_Profit_Loss': round(realized_pnl, 2),
        'Unrealized_Profit_Loss': round(unrealized_pnl, 2)
    }

def calculate_options_profits(transactions): #calculating profits of options

    transactions = transactions.sort_values('date')
    open_positions = []
    realized_pnl = 0

    for _, row in transactions.iterrows():
        action = row['action'].upper()
        quantity = row['quantity']
        price = row['price']
        amount = quantity * price
        description = str(row['description'])


        option_id = f"{row['symbol']} {description}"

        if action in ['BUY', 'BUY TO OPEN', 'SELL', 'SELL TO OPEN']:
            open_positions.append({
                'option_id': option_id,
                'quantity': quantity,
                'price': price,
                'amount': amount,
                'action': action,
                'description': description
            })
        elif action in ['SELL TO CLOSE', 'BUY TO CLOSE', 'EXPIRE']:
            remaining_qty = quantity

            while remaining_qty > 0 and open_positions:
                oldest_position = open_positions[0]


                if oldest_position['option_id'] != option_id:
                    break

                qty_to_close = min(remaining_qty, oldest_position['quantity'])

                if action == 'SELL TO CLOSE':

                    pnl = qty_to_close * (price - oldest_position['price'])
                elif action == 'BUY TO CLOSE':

                    pnl = qty_to_close * (oldest_position['price'] - price)
                elif action == 'EXPIRE':

                    if oldest_position['action'] in ['BUY', 'BUY TO OPEN']:
                        pnl = qty_to_close * (-oldest_position['price'])
                    else:
                        pnl = qty_to_close * oldest_position['price']

                realized_pnl += pnl
                remaining_qty -= qty_to_close
                oldest_position['quantity'] -= qty_to_close

                if oldest_position['quantity'] == 0:
                    open_positions.pop(0)


    unrealized_pnl = 0
    for position in open_positions:

        if position['action'] in ['BUY', 'BUY TO OPEN']:
            unrealized_pnl += -position['amount']
        else:
            unrealized_pnl += position['amount']

    return {
        'Realized_Profit_Loss': round(realized_pnl, 2),
        'Unrealized_Profit_Loss': round(unrealized_pnl, 2)
    }

def calculate_profits(df): #storing calculated profits

    df = df[df['transaction_category'] != 'Other']

    results = []

    stock_df = df[df['transaction_category'] == 'Stocks']
    for symbol, group in stock_df.groupby('symbol'):
        pnl = calculate_stock_profits(group)
        current_price = get_current_stock_price(symbol)

        results.append({
            'Symbol': symbol,
            'Transaction_Type': 'Stock',
            'Current_Price': round(current_price, 2),
            **pnl
        })

    options_df = df[df['transaction_category'] == 'Options']
    for (symbol, description), group in options_df.groupby(['symbol', 'description']):
        pnl = calculate_options_profits(group)

        results.append({
            'Symbol': f"{symbol} {description}",
            'Transaction_Type': 'Option',
            'Current_Price': 0,
            **pnl
        })

    return results

def filter_by_period(df, period): #filtering by period from the user

    today = datetime.now()

    if period == 'all':
        return df.copy()
    elif period == 'current_month':
        start_date = today.replace(day=1)
        return df[df['date'] >= start_date]
    elif period == 'last_month':
        start_date = (today.replace(day=1) - timedelta(days=1)).replace(day=1)
        end_date = today.replace(day=1)
        return df[(df['date'] >= start_date) & (df['date'] < end_date)]
    elif period == 'year_to_date':
        start_date = today.replace(month=1, day=1)
        return df[df['date'] >= start_date]
    else:
        return df.copy()

def display_summary_table(results, period): #displaying table

    if not results:
        print("No transactions found for the selected period")
        return

    df = pd.DataFrame(results)

    def format_currency(x):
        return f"${x:,.2f}" if isinstance(x, (int, float)) else x

    formatted_df = df.copy()
    for col in ['Current_Price', 'Realized_Profit_Loss', 'Unrealized_Profit_Loss']:
        if col in formatted_df.columns:
            formatted_df[col] = formatted_df[col].apply(format_currency)

    fig, ax = plt.subplots(figsize=(12, max(len(df) * 0.5, 5)))
    ax.axis('off')

    table = ax.table(
        cellText=formatted_df.values,
        colLabels=formatted_df.columns,
        cellLoc='center',
        loc='center'
    )

    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    period_names = {
        'all': 'All Time',
        'current_month': 'Current Month',
        'last_month': 'Last Month',
        'year_to_date': 'Year to Date'
    }
    plt.title(f"Transaction Summary - {period_names.get(period, period)}",
              fontsize=14, weight='bold', pad=20)

    plt.show()

def main(): #main function
    try:
        df = load_data('A_Schwab_Corp_Tx_2024_25.csv')
        clean_df = clean_data(df)
        clean_df['transaction_category'] = clean_df.apply(categorize_transaction, axis=1)
    except Exception as e:
        print(f"Error processing data: {e}")
        return

    period_dropdown = widgets.Dropdown(
        options=[
            ('All Time', 'all'),
            ('Current Month', 'current_month'),
            ('Last Month', 'last_month'),
            ('Year to Date', 'year_to_date')
        ],
        value='all',
        description='Time Period:',
        disabled=False,
    )

    output = widgets.Output()

    def update_display(change):
        with output:
            output.clear_output()
            period = change['new']
            filtered_df = filter_by_period(clean_df, period)
            results = calculate_profits(filtered_df)
            display_summary_table(results, period)

    period_dropdown.observe(update_display, names='value')

    display(period_dropdown)
    display(output)

    with output:
        initial_results = calculate_profits(clean_df)
        display_summary_table(initial_results, 'all')

if __name__ == "__main__":
    main()

Dropdown(description='Time Period:', options=(('All Time', 'all'), ('Current Month', 'current_month'), ('Last …

Output()